In [2]:
import re
import time
import json
import csv
import time
import nltk
import string
import numpy as np
import pandas as pd
from nltk import stem
from random import randint
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
ratings_dataFrame = pd.read_csv("ratings_dataFrame.csv", index_col=0, encoding= "ISO-8859-1")

books_dataFrame = pd.read_csv("books_dataFrame.csv", index_col=0,encoding= "ISO-8859-1")
books_dataFrame_new = pd.read_csv("books_dataFrame_new.csv", index_col=0,encoding= "ISO-8859-1")

users_dataFrame = pd.read_csv("users_dataFrame.csv", index_col=0, low_memory=False,encoding= "ISO-8859-1")


with open('userDict.json', 'r') as fp:
    
    userDict = json.load(fp)
    
with open('isbnDict.json', 'r') as fp:
    
    isbnDict = json.load(fp)
    
    
with open('featuresDict.json', 'r') as fp:
    
    featuresDict = json.load(fp)
    
    
with open('books.json', 'r') as fp:
    
    books = json.load(fp)
    
with open('users.json', 'r') as fp:
    
    users = json.load(fp)
    
    
with open('mostRated.json', 'r') as fp:
    
    f = json.load(fp)

In [7]:
features = set()

def featuresExtraction(books_dataFrame_new, features):
     
    years = books_dataFrame_new["Year-Of-Publication"].values
    features.update(set(years))

    authors = books_dataFrame_new["Book-Author"].values
    features.update(set(authors))

    publishers = books_dataFrame_new["Publisher"].values
    features.update(set(publishers))
    
    return features
    
    #for isbn in books_dataFrame_new.index:

     #   title = books_dataFrame_new.loc[isbn]["Book-Title"]

      #  for t in title.split(" "):

      #      features.add(t)

       # author = books_dataFrame_new.loc[isbn]["Book-Author"]
        #features.add(author)

        #publisher = books_dataFrame_new.loc[isbn]["Publisher"]
        #features.add(publisher)
        
    #return features

features = featuresExtraction(books_dataFrame_new, features)

featuresDict = {v:k for k,v in enumerate(features)}

# Compute feature vectors for items(books)

In [8]:
books = {k:{} for k in books_dataFrame_new.index }

for isbn in books:
    #add a weight to each information of a book 
    year = books_dataFrame_new.loc[isbn]["Year-Of-Publication"]
    books[isbn][year] =  0.1 
    
    author = books_dataFrame_new.loc[isbn]["Book-Author"]
    books[isbn][author] =  1
        
    publisher =  books_dataFrame_new.loc[isbn]["Publisher"]
    books[isbn][publisher] =  0.3
    
    
#    title = books_dataFrame_new.loc[isbn]["Book-Title"].strip().split(" ")
    
#    for word in title:
        
#        books[isbn][word] = books[isbn].get(word,0) + 1
    
    
#    author = books_dataFrame_new.loc[isbn]["Book-Author"]
#    books[isbn][author] = books[isbn].get(author,0) + 1
        
#    publisher =  books_dataFrame_new.loc[isbn]["Publisher"]
#    books[isbn][publisher] = books[isbn].get(publisher,0) + 1

# Compute feature vectors for users(readers)

In [11]:
#create a dictionary 
#key ----> user-id 
#values ----> dict(feature,"freq" or "weight")
users = {k:{} for k in userDict}


for isbn in books:
    
    try:
        for user in isbnDict[isbn]:
            #t is a rating 
            t = int(isbnDict[isbn][user])
            if t > 5:# take books rated more than 5
                
                for feature in books[isbn]:
                                      #feature -- This is the Key to be searched in the dictionary                                                              # Floor Division 
                    users[user][feature] =  users[user].get(feature,0) + books[isbn][feature] + t//5# 0 otherwise
        except:
        continue

# I want to recommend to a particular user  a generic book

# I start selecting users(w) that have rated > 9  books(v) rated > 8  by my user.

# After that I select books(z) rated > 9 by these users(w).

# The idea is to obtain a small subset (< 1000) of probably relevant users and/or items and using those to compute similarity (user/users or user/items).

# If use user/users you need to do another operation because from most similar users you have to select preferred books. The advantage is that the subset user/users (< 200) is smaller that in the case user/items(< 1000).

# Euristically user/items seems a better method with this feature vector.


# This is a Content Based method with buckets for users/items.


In [12]:
# select books rated by initial user
def genericContentBasedUserItems(user_number, userDict, isbnDict, featuresDict,users,books):
   
    #store in v-vector isbn of books rated by user_number with rating bigger than 8
    v = []

    for key in userDict[user_number]:
            #take books with rating bigger than 8
        if int(userDict[user_number][key]) > 8:

            v.append(key)

# select users that have rated books rated by initial users at the same time
    
    #Store in w-vector users with the same books quoted good by initial user
    w = []

    for isbn in v:
        for key in  isbnDict[isbn]:
                                 #take books with rating bigger than 8 
            if int(isbnDict[isbn][key]) > 8 and (key != user_number):

                w.append(key)   

    w = list(set(w))#take unique elements


    # books quoted by users that have rated the book(s) rated by initial user

    #storage in z books of users in w
    z = []

    for user in w:

        for isbn in userDict[user]:

            if int(userDict[user][isbn]) > 8:

                z.append(isbn)

    z = list(set(z))   
    
    X_myUser = np.zeros(len(featuresDict))
    #matrix n x 173878, n= len(z) described above
    X_items = np.zeros((len(z),len(featuresDict)))

    #fill  first matrix
    for feature in users[user_number]:

        X_myUser[featuresDict[feature]] = users[user_number][feature]#see inside users dictionary

    #fill second matrix
    for j in range(len(z)):

        try:
            for feature in books[z[j]]:


                X_items[j][featuresDict[feature]] = books[z[j]][feature] #see inside books dictionary

        except:
            continue
    # vectorized cosine similarity
    
    #matrix mult n x 173878, then sum the columns
    num = (X_myUser*X_items).sum(axis=1)
    #distance of vector X_myUser
    d_myUser = np.sqrt((X_myUser**2).sum())
    #distance of each vector in X_items
    d_items = np.sqrt((X_items**2).sum(axis=1))

    res = num/(d_myUser*d_items)
               #list of pairs(value cosinesim, book in w)
    f = sorted(zip(z,res), key=lambda tup: tup[1], reverse=True)

    f1 = [] #take a subset of f
    
    for tu in f:
            
            
        if tu[1] != 0.0 and not np.isnan(tu[1]):

            f1.append(tu)

    f1 = sorted(f1, key=lambda tup: tup[1], reverse=True)
            #take the first 10 recommendations, tu[0] is always an isbn
    recommendation = {user_number: [tu[0] for tu in f1[:10]]}
    
    return recommendation


def genericContentBasedUserUsers(user_number, userDict, isbnDict, featuresDict,users,books):
    
    #store in v-vector isbn of books rated by user_number with rating bigger than 8
    v = []

    for key in userDict[user_number]:

        if int(userDict[user_number][key]) > 8:

            v.append(key)

    # select users that have rated books rated by initial user        

    w = []

    for isbn in v:
        for key in  isbnDict[isbn]:

            if int(isbnDict[isbn][key]) > 8 and (key != user_number):

                w.append(key)    

    w = list(set(w))


    X_myUser = np.zeros(len(featuresDict))

    # others users
    X_otherUsers = np.zeros((len(w),len(featuresDict)))


    for feature in users[user_number]:

        X_myUser[featuresDict[feature]] = users[user_number][feature]


    for j in range(len(w)):

        try:
            for feature in users[w[j]]:

                X_otherUsers[j][featuresDict[feature]] = users[w[j]][feature]
        except:
            continue

    num = (X_myUser*X_otherUsers).sum(axis=1)

    d_myUser = np.sqrt((X_myUser**2).sum())
    d_otherUsers = np.sqrt((X_otherUsers**2).sum(axis=1))

    res = num/(d_myUser*d_otherUsers)
    
        #list of pairs(value cosinesim, book in w)
    f = sorted(zip(w,res), key=lambda tup: tup[1], reverse=True)

    Rated = userDict[user_number] 

    recommendation = {user_number: []}

    for tup in f:

        for book in userDict[tup[0]]:
    #if that book is rated more than 8 and it's not in books rated by my initial user and if there is not in list inside recomm
            if int(userDict[tup[0]][book]) > 8 and book not in Rated and book not in  recommendation[user_number]:

                recommendation[user_number].append(book)

            if len(recommendation[user_number]) > 10:

                break
            #stop loops--
        if len(recommendation[user_number]) > 10:

                break
                
    return recommendation
            

In [13]:
#print titles of recommended books generated by one of the functions above
def userRecommendations(user_number,books_dataFrame, recommendation): 
    
    for isbn in recommendation[user_number]:

        try:
            print(books_dataFrame.loc[isbn]["Book-Title"])
        except:
            continue
            
    return None

def userRatings(user_number, userDict,books_dataFrame):

    for isbn in userDict[user_number]:

        try:
            if  int(userDict[user_number][isbn]) > 8:
                print(books_dataFrame.loc[isbn]["Book-Title"])
        except:
            continue
            
    return None

In [ ]:
# this is not useful in this case

In [ ]:
d_user = sorted(userDict.keys())
d_isbn = sorted(isbnDict.keys())

In [14]:
def ContentBasedSimilarity(user_number,featuresDict,users,books,z):

    X_myUser = np.zeros(len(featuresDict))

    X_items = np.zeros((len(z),len(featuresDict)))


    for feature in users[user_number]:

        X_myUser[featuresDict[feature]] = users[user_number][feature]

    for j in range(len(z)):

        try:
            for feature in books[z[j]]:

                X_items[j][featuresDict[feature]] = books[z[j]][feature]
        except:
            continue

    num = (X_myUser*X_items).sum(axis=1)

    d_myUser = np.sqrt((X_myUser**2).sum())
    d_items = np.sqrt((X_items**2).sum(axis=1))

    res = num/(d_myUser*d_items)

    f = list(zip(z,res))
    
    return f


def mainContentBasedSimilarity(featuresDict,userDict,isbnDict,users,books,d_user,d_isbn,books_dataFrame,t):
    
    
    print("Please choose a user and/or an item\n")
    print("user or random user: \n")
    user_number = input()
    if user_number == "":   
        user_number = None
    print("item or random item: \n")
    book_numbers = input()
    if book_numbers == "":
        book_numbers = None

    if user_number != None or book_numbers != None: #case of a specific id-user o isbn book, given from input

        if user_number == None:

            n = randint(0,len(userDict))# random userid generator
            user_number = d_user[n]

        if book_numbers == None:

            m = randint(0,len(isbnDict))#random isbn generator
            book_numbers = d_isbn[m]
            
    else:
            
        n = randint(0,len(userDict))
        user_number = d_user[n]
        m = randint(0,len(isbnDict))
        book_numbers = d_isbn[m]

    z = book_numbers.split()

    ff = ContentBasedSimilarity(user_number,featuresDict,users,books,z)
    print("To user <", user_number, "> using a content based recommender system:\n")
    for f in ff:
        
        nan = np.isnan(f[1])
        print(f)

        if float(f[1]) > t and not nan :
            try:
                rec = books_dataFrame.loc[f[0]]["Book-Title"]
                print("we recommend <", rec,">")
            except:
                print("we recommend <", f[0],">")
        else:
            try:
                rec = books_dataFrame.loc[f[0]]["Book-Title"]
                print("we don't recommend <", rec,">")
            except:
                print("we don't recommend <", f[0],">")

In [15]:
def chooseUser(userDict, d_user):
    
    print("Please choose a user\n")
    print("user or random user: \n")
    user_number = input()
    if user_number == "":
        user_number = None
    if user_number == None:
        n = randint(0,len(userDict))
        user_number = d_user[n]
        return user_number
    else:
        return user_number

In [16]:
#put together all the function defined above
def mainContentBased(featuresDict,userDict,isbnDict,users,books,d_user,d_isbn,books_dataFrame,t):
    
    r = -1 # init  control variable
    while r not in [1,2]:
        
        print("\nPlease select a recommender system:\n")
        print("1 Content Based user-items\n")
        print("2 Content Based user-users\n") #need to choose  option 1 or 2
        try:
            r = int(input())
        except:
            continue
    
    if r == 1:
        user_number = chooseUser(userDict,d_user)
        recommendation = genericContentBasedUserItems(user_number, userDict, isbnDict, featuresDict,users,books)
        if recommendation[user_number] == []:
            print("We don't have a recommendation for you. We can only recommend the most popular and well rated books in the database:\n")
            for i in f: 
                print(books_dataFrame["Book-Title"][i[0]])
            return None
        print("For user <",user_number,"> we recommend using a content based approach:\n")
        userRecommendations(user_number,books_dataFrame, recommendation)
    elif r == 2:
        user_number = chooseUser(userDict,d_user)
        recommendation = genericContentBasedUserUsers(user_number, userDict, isbnDict, featuresDict,users,books)
        if recommendation[user_number] == []:
            print("We don't have a recommendation for user <",user_number,">. We can only recommend the most popular and well rated books in the database:\n")
            for i in f:
                print(books_dataFrame["Book-Title"][i[0]])
            return None
        print("For user <",user_number,"> we recommend using a content based approach:\n")
        userRecommendations(user_number,books_dataFrame, recommendation)

In [21]:
r = -1
while True:
    while r not in ["y","n"]:
        print("\nDo you want a recommendation?\n")
        print("y")
        print("n\n")
        r = input()
    if r == "y":
        mainContentBased(featuresDict,userDict,isbnDict,users,books,d_user,d_isbn,books_dataFrame,0.2)
        
        try:
            mainContentBased(featuresDict,userDict,isbnDict,users,books,d_user,d_isbn,books_dataFrame,0.2)
        except:
            print("I you are a new user (also known as Cold Star). We can only recommend to you the most popular books in our database:\n")
            for i in f:# f is the return of  mostPopularBooksRecommendation
                print(books_dataFrame["Book-Title"][i[0]])
        r = -1
    else:
        print("Bye")
        break


Do you want a recommendation?

y
n

y

Please select a recommender system:

1 Content Based user-items

2 Content Based user-users

1
Please choose a user

user or random user: 

147839
We don't have a recommendation for you. We can only recommend the most popular and well rated books in the database:

Harry Potter and the Order of the Phoenix (Book 5)
The Hobbit : The Enchanting Prelude to The Lord of the Rings
To Kill a Mockingbird
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Chamber of Secrets (Book 2)
The Da Vinci Code
The Catcher in the Rye
The Five People You Meet in Heaven
The Lovely Bones: A Novel
Fahrenheit 451

Please select a recommender system:

1 Content Based user-items

2 Content Based user-users

2
Please choose a user

user or random user: 

147839
We don't have a recommendation for user < 147839 >. We can only recommend the most popular and well rated books in the database:

Harry Potter and the Order of the Phoenix (Book 5)
T

In [63]:
def mostPopularBooksRecommendation(isbnDict):
    
    mean_ratings_isbn = {k:[] for k in isbnDict} #dictionary of list of ratings given to a book 
    for isbn in mean_ratings_isbn:
        for user in isbnDict[isbn]:
            mean_ratings_isbn[isbn].append(int(isbnDict[isbn][user]))#add each rating given to a book
            
    f = [] #f is a list of list
    for isbn in mean_ratings_isbn:
                        #len of rating obtained
        f.append([isbn,len(mean_ratings_isbn[isbn]),np.mean(mean_ratings_isbn[isbn])])
    f = sorted(f, key=lambda tup: tup[1], reverse=True)
    f = f[:100] #take the first 100 with the most amount of votes
    f = sorted(f, key=lambda tup: tup[2], reverse=True)
        #from those 100 take just 10 with the the highest average
    f = f[:10]
    
    for i in f:
        
        print(books_dataFrame["Book-Title"][i[0]])
    
    
    return f

In [64]:
mostPopularBooksRecommendation(isbnDict)

Harry Potter and the Order of the Phoenix (Book 5)
The Hobbit : The Enchanting Prelude to The Lord of the Rings
To Kill a Mockingbird
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Harry Potter and the Chamber of Secrets (Book 2)
The Da Vinci Code
The Catcher in the Rye
The Five People You Meet in Heaven
The Lovely Bones: A Novel
Fahrenheit 451


[['043935806X', 334, 5.5718562874251498],
 ['0345339681', 281, 5.0071174377224201],
 ['0446310786', 389, 4.9203084832904889],
 ['059035342X', 571, 4.9001751313485116],
 ['0439064872', 351, 4.7293447293447297],
 ['0385504209', 883, 4.6523216308040771],
 ['0316769487', 403, 4.6352357320099253],
 ['0786868716', 427, 4.543325526932084],
 ['0316666343', 1295, 4.4687258687258691],
 ['0345342968', 321, 4.40809968847352]]